# **Workshop Machine Learning Nivel**

---

## Benodigde onderdelen om te beginnen



###In dit stukje worden de noodzakelijke bibliotheken geinstalleerd die nodig zijn voor data verwerking, visualisatie en machine learning


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

## Hier wordt de dataset ingeladen



In [ ]:
df = pd.read_csv('Datafile.csv')


---

# **Comments voor de verwerking**

---





## Extra informatie:

### 1. Een patient met hartfalen wordt als 1 aangeduid, een patient zonder hartfalen als 0.
### 2. De aanwezigheid van hartfalen wordt 'target' genoemd
### 3. Er zijn soms 'extra vragen' toegevoegd: Dit is voor verdere verdieping mocht je tijd over hebben

---
 
#**Deel 1 - Data Preparatie**

---



## De eerste stap is altijd het bekijken van de data. Hier doen we dit door:
### 1). De tabel zelf te bekijken
### 2). De verdeling cases / controls te bekijken
### 3). Scatterplots van variabelen te bekijken
### 4). Histogrammen van data te bekijken


In [ ]:
# Bekijk je tabel door het runnen van deze cel
df

In [ ]:
# Bekijk de verdeling cases: Controls
df['target'].sum() / df['target'].count()

In [ ]:
# Visualiseer relaties tussen variabelen
# 2D scatterplot of two variables
px.scatter(df, x='Leeftijd (continue)', y='BMI', color = 'target')

##`Vraag 1: Wat valt je op?`




In [ ]:
# Opdracht: Pas onderstaande code aan om zelf een 2d scatterplot te maken tussen twee variabelen
px.scatter(df, x='', y='', color = 'target')

In [ ]:
# Maak een histogram van een variabele gesplitst op de outcome. Je kan de code aanpassen om andere variabelen te tonen. Wat zie je?
import plotly.graph_objects as go
import plotly.figure_factory as ff

fig = go.Figure()
fig.add_trace(go.Histogram(x=df.loc[df['target']==0,'Leeftijd (continue)'].values, histnorm='percent'))
fig.add_trace(go.Histogram(x=df.loc[df['target']==1,'Leeftijd (continue)'].values, histnorm='percent'))

### Het is zichtbaar dat datasoorten verschillen in de dataset (continu vs noncontinu), de ranges sterk verschillen en er een flinke imbalans in de ratio case:controls is. Belangrijker echter is dat er duidelijk artefacten in de data zitten (cellen met de waarde 999), evenals lege cellen (cellen met de waarde NaN). Dit kunnen we bekijken en verhelpen

In [ ]:
# Toon per variabele het aantal artifacten. Neem ook in gedachten hoeveel patienten er totaal waren. 
(df==999).sum()

#### En voor de lege cellen:

In [ ]:
# Aantal lege cellen per parameter. 
df.isnull().sum()

##`Vraag 2: Met welke variabelen kan je nog niet aan de slag zonder voorbewerking?`

## Dat zijn een redelijk aantal missings en artifacten! Omdat artifactverwijdering en NaN imputatie standaard operaties zijn in statistiek, gaan we ze hier niet uitgebreid bespreken. Hier worden de artifacten vervangen door NaNs, waarna alle NaNs geimputeerd worden door de mean (continue waarde) of median (non-oontinue waarde).

In [ ]:
# Artifacten naar NaNs omzetten
mask = df==999
df[mask] = np.nan

In [ ]:
# Aantal NaNs aanwezig
df.isnull().sum()

In [ ]:
# Imputatie. Hier hoef je niet perse diep op in te gaan; Het kan wel interessant zijn
from sklearn.impute import SimpleImputer

# Select which parameters are continuous
non_continuous = ['Chronische aandoeningen', 'Geslacht','Polyfarmacie','rGFR','AF', 'Alcoholmisbruik','COVID doorgemaakt','COPD','Slaapproblemen']
continuous = [x for x in df.columns if x not in non_continuous]

# Impute continuous values based on the mean
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(df[continuous])
df[continuous] = imp_mean.transform(df[continuous])

# Impute non-continuous values based on the mode
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(df[non_continuous])
df[non_continuous] = imp_mode.transform(df[non_continuous])

# Some values should be rounded
to_round = ['Aantal consulten huisarts jaar ervoor', 'Leeftijd (continue)']
for param in to_round:
  df[param] = np.round(df[param])

In [ ]:
## Opdracht: Kijk opnieuw naar de scatterplot. Probeer ook andere grafieken te maken om je data te valideren. Zijn de artifacten verdwenen?
px.scatter(df, x='Leeftijd (continue)', y='Pro-BNP (pg/ml)', color = 'target')

##`Vraag 3: Heb je al een idee welke variabelen voorspellend zouden kunnen zijn voor je uitkomst?`



## Preprocessing is klaar: Hoewel dit hier snel is gedaan omdat de data al goed verzorgd is, is dit meestal de belangrijkste stap in het ontwikkelen van een ML model. Onthoud: Garbage in is garbage out. Een veel gemaakte fout is het blind toepassen van ML technieken, wat dan leidt tot slechte of onjuiste resultaten. Daarom: Mocht je dus een model ontwikkelen, leg een sterke focus op de data preprocessing.

##`Extra vraag: De preprocessing is nu grof gedaan. Zou je dit anders aanpakken, en zo ja, hoe?`

---

#**Deel 2 - Machine learning modellen bouwen en interpreteren**

---


###We gaan verder met ML modellen bouwen. Waarom willen we eigenlijk ML modellen bouwen, en niet een simpele logistische regressie? Dit komt doordat complexere modellen complexere (non-lineaire) verbanden mee kunnen nemen.
###We beginnen met een decision tree, en nemen steeds ook logistische regressie mee. Een decision tree is vergelijkbaar met een klinische beslisboom: Bij iedere stap is er een indien dit, dan dit, anders dat (e.g. if age <20, ga links, anders, ga rechts). Zo wordt er een boom gebouwd, met aan de onderkant een classificatie.



##`Vraag 4: Heb je een idee wat de uitkomsten (accuracy) van de beslisboom en de logistische regressie zullen / kunnen zijn? Waarom?`

In [ ]:
# Hier wordt een logistische regressie en een beslisboom gemaakt

# Importeer de benodigde packages
#from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# We geven handmatig aan welke parameters we mee willen nemen voor trainen. Dit is niet perse de meest efficiente manier, maar toont het verschil goed aan.
parameters_toInclude = ['Pro-BNP (pg/ml)','BMI','Aantal consulten huisarts jaar ervoor','Chronische aandoeningen',
            'Leeftijd (continue)', 'Geslacht','Polyfarmacie','rGFR','AF', 'Alcoholmisbruik','COVID doorgemaakt','COPD','totaal cholesterol (mmol/l)','Slaapproblemen','target']

#------------------ Logistische regressie -------------------------------------#
# Geef aan welk model er wordt gebruikt
print('Start met trainen van het LR model')
model = LogisticRegression(max_iter=2000)

# Bij de training moet je de input data aangeven (waar je op traint), evenals de uitkomst data (de labels, hier 'target' in het dataframe)
# We geven hier dus het volledige dataframe als input, en de labels als target
model.fit(df[parameters_toInclude], df['target'])
print('LR model is getraind')

# Score het model
modelscore = model.score(df[parameters_toInclude], df['target'])
print('Het LR model heeft een accuratesse van: {}\n'.format(modelscore))


# ------------------ Decision tree --------------------------------------------#
# Geef aan welk model er wordt gebruikt
print('Start met trainen van de decision tree')
model = DecisionTreeClassifier()

# Bij de training moet je de input data aangeven (waar je op traint), evenals de uitkomst data (de labels, hier 'target' in het dataframe)
# We geven hier dus het volledige dataframe als input, en de labels als target
model.fit(df[parameters_toInclude], df['target'])
print('De decision tree is getraind')

# Score het model
modelscore = model.score(df[parameters_toInclude], df['target'])
print('De decision tree heeft een accuratesse van: {}\n'.format(modelscore))

##`Vraag 5: Wat komt er uit de analyse? Komt dit overeen met je verwachtingen? Waarom wel en waarom niet, en wat kunnen we eraan doen? Zou je een LR model gebruiken of een decision tree?`

-> In de verborgen code meer uitleg: Klik daarop

In [ ]:
# Het probleem is dat we de volledige dataframe als input hebben gegeven: Hier zit ook de target in. 
#Het model kan heel makkelijk deze 1:1 relatie eruit halen, en dus altijd goed voorspellen. 
#Dit geeft aan hoe makkelijk een foutje gemaakt kan worden, en wat voor effecten dit kan hebben. 

# Laten we dit rechtzetten, en nu trainen zonder de input erin te stoppen.

### De aanpassing die hierboven beschreven staat voeren we hier door

In [ ]:
# We geven handmatig aan welke parameters we mee willen nemen voor trainen. Dit is niet perse de meest efficiente manier, maar toont het verschil goed aan.
parameters_toInclude = ['Pro-BNP (pg/ml)','BMI','Aantal consulten huisarts jaar ervoor','Chronische aandoeningen',
            'Leeftijd (continue)', 'Geslacht','Polyfarmacie','rGFR','AF', 'Alcoholmisbruik','COVID doorgemaakt','COPD','totaal cholesterol (mmol/l)','Slaapproblemen']

#------------------ Logistische regressie -------------------------------------#
# Geef aan welk model er wordt gebruikt
print('Start met trainen van het LR model')
model = LogisticRegression(max_iter=2000)

# Bij de training moet je de input data aangeven (waar je op traint), evenals de uitkomst data (de labels, hier 'target' in het dataframe)
# We geven hier dus niet meer het label mee als input
model.fit(df[parameters_toInclude], df['target'])
print('Model is getraind')

# Score het model
modelscore = model.score(df[parameters_toInclude], df['target'])
print('Het model heeft een accuratesse van: {}\n'.format(modelscore))

# ------------------ Decision tree --------------------------------------------#
# Geef aan welk model er wordt gebruikt
print('Start met trainen van de decision tree')
model = DecisionTreeClassifier()

# Bij de training moet je de input data aangeven (waar je op traint), evenals de uitkomst data (de labels, hier 'target' in het dataframe)
# We geven hier dus niet meer het label mee als input
model.fit(df[parameters_toInclude], df['target'])
print('Decision tree is getraind')

# Score het model
modelscore = model.score(df[parameters_toInclude], df['target'])
print('De decision tree heeft een accuratesse van: {}\n'.format(modelscore))


### Het nieuwe model is nu niet getraind met per ongeluk de labels erin.

##`Vraag 6: Wat vind je nu van de accuracy van beide modellen? Waarom?`


### Het is goed om op te merken dat de case: control ratio 0.1 is... Oftewel, een sterke imbalance.

##`Vraag 7: Als je dit meeneemt, wat vind je nu van de accuracy? Kun je de uitkomst verklaren? En is accuracy dan nog wel een goede maat?`

### Om dit eerlijker te kunnen bekijken, introduceren we de ROC curve en de AUC erbij. Hierbij is de optimale waarde van de AUC 1, en de ROC curve is optimaal als hij in de linkerbovenhoek plakt.

In [ ]:
from sklearn.metrics import roc_auc_score, RocCurveDisplay

#------------------ Logistische regressie -------------------------------------#
# Geef aan welk model er wordt gebruikt
print('Start met trainen van het LR model')
model = LogisticRegression(max_iter=2000)

# Bij de training moet je de input data aangeven (waar je op traint), evenals de uitkomst data (de labels, hier 'target' in het dataframe)
# We geven hier dus niet meer het label mee als input
model.fit(df[parameters_toInclude], df['target'])
print('Model is getraind')

# Maak een predictie op de data met het model
prediction = model.predict_proba(df[parameters_toInclude])[:, 1]

# Toon de ROC curve
RocCurveDisplay.from_predictions(df['target'], prediction)
print('AUC van de ROC curve met het LR model: {}'.format(roc_auc_score(df['target'], prediction)))

# ------------------ Decision tree --------------------------------------------#
# Geef aan welk model er wordt gebruikt
print('Start met trainen van de decision tree')
model = DecisionTreeClassifier()

# Bij de training moet je de input data aangeven (waar je op traint), evenals de uitkomst data (de labels, hier 'target' in het dataframe)
# We geven hier dus niet meer het label mee als input
model.fit(df[parameters_toInclude], df['target'])
print('Decision tree is getraind')

# Maak een predictie op de data met het model
prediction = model.predict_proba(df[parameters_toInclude])[:, 1]

# Toon de ROC curve en de AUC
RocCurveDisplay.from_predictions(df['target'], prediction)
print('AUC van de ROC curve met de decision tree: {}'.format(roc_auc_score(df['target'], prediction)))

##`Vraag 8: Wat vind je van deze resultaten? Zijn ze logisch? Welk model vind jij dat de beste voorspellende waarde heeft?`



### We weten nu hoe ons model presteert. Dan willen we dit testen op nieuwe patienten. Vanwege de resultaten vervolgen we dit enkel met de decision tree, omdat duidelijk is wat ML kan toevoegen.

In [ ]:
# Laad de nieuwe patienten:
df_test = pd.read_csv('TestPatienten2.csv')

# Test de score en AUC voor de nieuwe patienten:
modelscore = model.score(df_test[parameters_toInclude], df_test['target'])
print('Het model heeft een accuratesse op de nieuwe patienten van: {}'.format(modelscore))

prediction = model.predict_proba(df_test[parameters_toInclude])[:, 1]
print('AUC van de ROC curve op de nieuwe patienten: {}'.format(roc_auc_score(df_test['target'], prediction)))

##`Vraag 9: Wat vind je van deze resultaten op de test set? Zijn ze logisch?`

##`Extra vraag: Probeer eens om een logistisch regressie model te maken, en deze te testen op de test set. Kun je de uitkomsten hiervan verklaren, ook in relatie tot de uitkomsten van de beslisboom?`

---

#**Deel 3 - Crossvalidatie en overfitten**

---

### We zagen in het vorige gedeelte dat de data heel goed op de training set werkte, maar veel minder op de test set. Wellicht heeft de beslisboom dus te specifiek op de training set gefit. Laten we eens naar de beslisboom kijken.

In [ ]:
from sklearn import tree
# Geef aan welk model er wordt gebruikt
print('Start met trainen van de decision tree')
model = DecisionTreeClassifier()

# Bij de training moet je de input data aangeven (waar je op traint), evenals de uitkomst data (de labels, hier 'target' in het dataframe)
# We geven hier dus niet meer het label mee als input
model.fit(df[parameters_toInclude], df['target'])
print('Decision tree is getraind')

# Exporteer de figuur van de boom naar de Tree-full.png file
import graphviz 
import pydot

dot_data = tree.export_graphviz(model, out_file='Tree_full.dot', 
                                feature_names=parameters_toInclude,  
                                class_names=['Gezond','Patient'],
                                filled=True)

(graph_tosave,) = pydot.graph_from_dot_file('Tree_full.dot')
graph_tosave.write_png('Tree_full.png')

# Plot de boom zelf
dot_data = tree.export_graphviz(model, out_file=None, 
                                feature_names=parameters_toInclude,  
                                class_names=['Gezond','Patient'],
                                filled=True)

graph = graphviz.Source(dot_data, format="png")
graph

##`Vraag 10: Laten we eens kijken naar de beslisboom zelf: Hoe specifiek heeft deze gefit? (let op: het kan even duren voordat deze zichtbaar wordt: Je kan ook de Tree_full.png downloaden na het runnen om op je pc zelf te kijken). Is hier sprake van te specifieke training naar jouw mening?`

### Een gevaar van overfitting is dat je een hele goede performance op je training set hebt, maar het model niet 'generaliseerbaar' is: Oftewel, het presteert op nieuwe patienten niet net zo goed.

### Het is daarom belangrijk om te weten hoe je model het gaat doen op nieuwe data; Dan weet je de 'echte' performance. Daarom hebben we crossvalidatie nodig. We splitsen de dataset in 5 stukken (folds), waarbij we trainen op 4 folds, en testen op 1 fold. De testfold is niet meegenomen in de training, en is dus 'nieuwe' data voor het model. Dit wordt 5x gedaan: Iedere keer met een andere testfold.

### Voor de volledigheid wordt dit hier gedaan met een 'stratified' crossvalidatie. Dit zorgt ervoor dat in iedere fold de ratio cases:controls gelijk blijft.

In [ ]:
# Importeer benodigde packages
from sklearn.model_selection import StratifiedKFold

# Maak de crossvalidatie
skf = StratifiedKFold(n_splits=5)

# Overig, niet belangrijk
count=1

# Parameters om mee te nemen
parameters_toInclude = ['Pro-BNP (pg/ml)','BMI','Aantal consulten huisarts jaar ervoor','Chronische aandoeningen',
            'Leeftijd (continue)', 'Geslacht','Polyfarmacie','rGFR','AF', 'Alcoholmisbruik','COVID doorgemaakt','COPD','totaal cholesterol (mmol/l)','Slaapproblemen']

# De dataframe en de target wordt hier alvast met de juiste parameters gedefinieerd, omdat dat nodig is (niet belangrijk)
df_onlyGoodParam = df[parameters_toInclude]
target = df['target']


# Deze loop loopt selecteert iedere keer een andere split.
# De train_index zijn alle patienten in de training set.
# De test_index zijn alle patienten in de test set.
# Voor iedere split wordt een model gemaakt, en de performance berekent (zoals hierboven al eerder gedaan).

for train_index, test_index in skf.split(df_onlyGoodParam, target):
  print('--------------- SPLIT NUMMER {} -------------------------'.format(count))

    # Geef aan welk model er wordt gebruikt
  print('Start met trainen van de decision tree')
  model = DecisionTreeClassifier()

  # Bij de training moet je de input data aangeven (waar je op traint), evenals de uitkomst data (de labels, hier 'target' in het dataframe)
  # We geven hier dus niet meer het label mee als input
  # De training index (de train set) wordt als input gegeven.
  model.fit(df_onlyGoodParam.iloc[train_index,:], target.iloc[train_index])
  print('Decision tree is getraind')

  # Performance op de training folds:
  modelscore = model.score(df_onlyGoodParam.iloc[train_index,:], target.iloc[train_index])
  print('Accuratesse op de training folds: {}'.format(modelscore))

  prediction = model.predict_proba(df_onlyGoodParam.iloc[train_index])[:, 1]
  print('AUC van de ROC curve op de training folds: {}'.format(roc_auc_score(target.iloc[train_index], prediction))) 

  # Performance op de test folds:
  modelscore = model.score(df_onlyGoodParam.iloc[test_index,:], target.iloc[test_index])
  print('Accuratesse op de test folds: {}'.format(modelscore))

  prediction = model.predict_proba(df_onlyGoodParam.iloc[test_index])[:, 1]
  print('AUC van de ROC curve op de test folds: {}\n'.format(roc_auc_score(target.iloc[test_index], prediction))) 

  # Overig
  count = count+1


##`Vraag 11: Wat zie je in de performance tussen de folds? En in iedere fold tussen de train en test set? Kun je dit verklaren?`

### Een manier om deze overfitting aan te pakken is om de decision tree minder diep te laten trainen. Dus oftewel: De decision tree die we zagen, af te kappen na een diepte van x keer splitten. Probeer dit eens uit door in de bovenstaande code in de syntax 'DecisionTreeClassifier()' tussen de haakjes max_depth=x in te voegen, met de x een waarde naar je keuze. Probeer ook eens verschillende waardes van x uit. 

##`Vraag 12: Wat gebeurt er? Kun je dit verklaren? Als je wilt kun je onderstaande syntax runnen om de laatste beslisboom die je trainde weer te inspecteren.`

In [ ]:
from sklearn import tree

# Exporteer de figuur van de boom naar de Tree_part.png file
dot_data = tree.export_graphviz(model, out_file='Tree_part.dot', 
                                feature_names=parameters_toInclude,  
                                class_names=['Gezond','Patient'],
                                filled=True)

(graph_tosave,) = pydot.graph_from_dot_file('Tree_part.dot')
graph_tosave.write_png('Tree_part.png')

# Plot de boom zelf
dot_data = tree.export_graphviz(model, out_file=None, 
                                feature_names=parameters_toInclude,  
                                class_names=['Gezond','Patient'],
                                filled=True)

graph = graphviz.Source(dot_data, format="png")
graph

##`Vraag 13: Nu we een aangepast model hebben, en het overfitting iets hebben tegengegaan, hoe presteert dit op de nieuwe patienten? Hiervoor laden we opnieuw de dataset in, en testen we het wederom. Hoe doet het het nu?`

In [ ]:
# Laad de nieuwe patienten:
df_test = pd.read_csv('TestPatienten2.csv')

# Test de score en AUC voor de nieuwe patienten:
modelscore = model.score(df_test[parameters_toInclude], df_test['target'])
print('Het model heeft een accuratesse op de nieuwe patienten van: {}'.format(modelscore))

prediction = model.predict_proba(df_test[parameters_toInclude])[:, 1]
print('AUC van de ROC curve op de nieuwe patienten: {}'.format(roc_auc_score(df_test['target'], prediction)))

##`Extra vraag: Wat denk je dat de invloed van een verschillend aantal folds op de performance is, en waarom? Je kan dit uittesten door de n_splits te veranderen (nu 5). Komt dit overeen met je verwachtingen?`

---

# **Deel 4: Hyperparameter tuning en feature selectie**

---

### Wat je in het vorige gedeelte aan het einde deed (het aanpassen van de diepte van de decision tree), was het aanpassen van een zogeheten hyperparameter. Dit zijn modelinstellingen die bepalen hoe een model traint, in dit geval hoe diep een beslisboom traint. Vrijwel ieder model heeft bergen van deze instellingen, en je weet van tevoren niet wat de beste zijn. Wat daarom vaak wordt gedaan is 'hyperparameter tuning'. Hierbij wordt een grote hoeveelheid combinaties van hyperparameters uitgeprobeerd, en wordt simpelweg het beste model gekozen. 

### Voorbeelden voor de decision tree zijn zijn:
#####max_depth=x -> De maximale diepte
#####class_weight = 'balanced' of None -> of er een correctie moet komen voor de imbalans in case:controls. Is dat handig hier?
#####min_samples_leaf = x -> Hoeveel samples er minimaal per uiteindelijk 'blad' aan de boom moeten zitten

### Wanneer je meerdere hyperparameters invoert, zorg dat je deze met een komma scheidt: Dus max_depth=20 , class_weight = 'balanced' (bijvoorbeeld)

Op de pagina https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier vind je uitgebreide mogelijkheden mocht je interesse hebben

##`Vraag 14: Ga eens terug naar de code waar je de hyperparameters veranderde, en probeer eens wat meerdere variaties uit. Kun je een betere performance halen op de validatie sets? En hoe presteert dit dan op de test set? Kun je dit verklaren?`

### Ter info: Meestal worden deze parameters binnen een fold gezocht, en dan de beste gevonden parameter getest op de test fold. Hier bestaan standaard algoritmes voor (GridSearchCV in sklearn), zodat dit niet handmatig hoeft te worden geprogrammeerd. Dit valt echter buiten de scope van deze workshop

## Feature selectie
### Wat we zagen is een totaal van 8000 patienten, en 15 features. Hierdoor kan het model redelijk goed trainen, ook al zijn niet alle features even belangrijk. Dit wordt anders wanneer het aantal features (bijna) even groot is als het aantal patienten: Dan kunnen patronen door kans een voorspellende waarde hebben

### Een oplossing hiervoor is feature selectie. Er zijn vele manieren, maar een manier is forward feature selectie. Hierbij wordt steeds een parameter aan het model toegevoegd, tot alle parameters op zijn. Bij iedere stap wordt de performance berekend, waardoor je ziet wanneer meer features toevoegen niet meer helpt.

### Hieronder wordt forward feature selectie geimplementeerd en getest op de AUC. Ook wordt de performance per aantal parameters geplot

In [ ]:
# Run deze cel 1x, dan werkt het feature selectie algoritme
!pip install mlxtend --upgrade

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Train een forward feature selection algoritme. Dit algoritme begint met een leeg model. Het maakt een crossvalidatie, en voegt dan iedere parameter toe als test (dus als je 15 features hebt, maakt het algoritme 15 modellen met crossvalidatie, ieder met 1 parameter in dit model).
# De beste feature gaat in het model, en dan probeert het wederom de rest van de parameters toe te voegen (dus weer 14 testen).
# Dit doet het tot alle features op zijn: Het model met het aantal features dat optimaal is wordt dan geselecteerd.
# Let wel: Er is een maximale diepte ingesteld voor de decision tree van 6. Als je de parameters in je model gebruikt, moet dit voor het beste resultaat ook de hyperparameter zijn.

sfs = SFS(DecisionTreeClassifier(max_depth=6), k_features=14, forward=True,cv=5, scoring = 'roc_auc')
sfs.fit(df.iloc[:,:-1], target)

In [ ]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt
fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')

plt.title('Sequential Forward Selection ( met SE)')
plt.grid()
plt.show()

### Belangrijk om te weten is per stap de parameters die zijn ingevoegd. Die kun je aflezen door de volgende code te runnen. De feature_names bij {4} stelt bijvoorbeeld de 4 geincludeerde parameters voor bij bovenstaande grafiek op x=4 

In [ ]:
# Run om de parameters per stap af te lezen
sfs.get_metric_dict()

##`Vraag 15: Hoeveel parameters moet je toevoegen voor een optimaal model? Welke parameters zijn dit? Ga eens terug naar je model, en gebruik alleen deze parameters voor het model. Veranderd de performance, en kun je dit verklaren?`


---

# **Verdere verdieping**

---

Dit waren de dingen die we graag wilden bespreken in de workshop. Je hebt hopelijk meegenomen:

1). Het belang van preprocessing.

2). De basis van machine learning modellen.

3). Dingen waar je op moet letten bij ML modellen. Denk hierbij ook aan test en train sets!

4). Kennis over crossvalidatie en overfitting.

5). Belangrijke additionele kennis (hyperparameters, feature selectie).

### Mocht je nog verder interesse hebben, kunnen we je de volgende stappen aanraden:

1). Probeer bovenstaande analyse eens met een random forest. Dit is als het ware een hoop beslisbomen bij elkaar. Verbetert dit het model?

2). Op www.kaggle.com zijn veel datasets te vinden waarmee je kan spelen. Hierbij is ook vaak uitleg aanwezig. Kijk ook eens naar https://www.kaggle.com/learn/intro-to-machine-learning. Extraatje: Als je de beste bent ter wereld, kun je er grote prijzen mee verdienen ;)

